# Visualising Player Contributions and Ratings

This notebook uses Plotly to create interactive visualisations to analyse player stats and contributions. The following visualisations will be created:
1. **Donut Chart**: Average contribution to team by position.
2. **Line Graph**: Visualised player ratings.
3. **Heatmap**: Average key stats by position.


In [28]:
# Import libraries
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [29]:
# Load data from database
conn = sqlite3.connect('Oxford_Utd_Stats.db')
df = pd.read_sql_query("SELECT * FROM calculations", conn)  # Load the calculations table
conn.close()

df.head()   # Display first few rows of dataframe


,Name,Position,raw_score,normalised_rating
0,Mark Harris,CF,9.209854,4.177983
1,Dane Scarlett,CF,7.205264,3.373266
2,Will Goodwin,CF,1.565622,1.109305
3,Marselino Ferdinan,AM,3.446880,1.864511
4,Przemysław Płacheta,LW,7.605424,3.533905


## 1. Donut Chart: Average Contribution by Position
The following chart represents the average contribution of each position to the team's performance, based on their normalised ratings.

Highlights which positions tend to perform better on average, showcasing the relative importance of each position to the team's success.

### Results:
- Defensive Midfielders (**DM**) contribute the most (16%) to the team's performance, indicating their role in balancing defense and attack.
- Attacking Midfielders (**AM**) follow with 12.4%, showcasing their importance in creating goal scoring opportunities and link up play.
- Centre Backs (**CB**) and Left Backs (**LB**) contribute significantly (around 11-12%) highlighting the strength of the defensive line, Right Backs (**RB**) didn't perform as well suggesting the right side might be weaker than the left - something teams may target when coming up against Oxford Utd.
- Forwards (**CF**, **LW**, **RW**) have relatively smaller contributions, reflecting a possible dependence on defensive and midfield roles for overall success.

In [30]:
average_contribution_per_position = df.groupby('Position')['normalised_rating'].mean().reset_index()    # Calculate average contribution per position
average_contribution_per_position = average_contribution_per_position.rename(columns={'normalised_rating': 'average_rating'})   # Rename column

fig = px.pie(   # Create a donut chart (pie chart becomes donut chart by using hole=0.4)
    average_contribution_per_position,
    names='Position',
    values='average_rating',
    title='Average Contribution by Position',
    hole=0.4  # Donut chart
)

fig.update_traces(textinfo='percent+label')  # Show percentage and label
fig.update_layout(
    title_font=dict(size=24, family='Arial'),
    title_x=0.5,  # Title centred
    title_y=0.9,
    font=dict(size=14),
    showlegend=False    # Hide legend
)

fig.show()


## 2. Line Graph: Players' Ratings Visualised

This graph displays each player's normalised rating, allowing comparison of individual performances.

In [31]:
sorted_df = df.sort_values(by="normalised_rating", ascending=False) # Sort the dataframe by normalised rating

fig_line = px.line( # Create a line graph to show normalised ratings of players
    sorted_df,
    x="Name",
    y="normalised_rating",
    title="Players' Normalised Ratings",
    labels={"normalised_rating": "Normalised Rating", "Name": "Player Name"},
    markers=True,
)

fig_line.update_layout(
    title_font_size=20,
    xaxis_tickangle=270,  # Player names at 270 degrees for optimal visualisation
    xaxis_title="Player Name",
    yaxis_title="Normalised Rating",
    yaxis=dict(range=[1, 10]),  # Rating scale 1-10
    showlegend=False  # Hide legend
)

fig_line.show()


## 3. Heatmap: Average Key Stats by Position

This heatmap provides a visual representation of the average values of key performance statistics for each position. 

### This Graph Shows:
Performance of players in different positions based on six key statistics:
- **Goals**
- **Assists**
- **Tackles**
- **Interceptions**
- **Successful Dribbles**
- **Shots on Target**

Each cell represents average value of the corresponding statistic for a given position. Colour intensity indicates the magnitude of the value, with brighter colours (e.g. yellow) representing higher values and darker colours (e.g. purple) representing lower values. 

### Results:
- Defensive Midfielders (**DM**) excel in **tackles** and **interceptions**, reflecting crucial defensive responsibilities.
- Forwards (**CF**, **LW**, **RW**) contribute significantly in **goals**, **assists**, and **successful dribbles**, showcasing their attacking influence.
- Central Midfielders (**CM**) balance both defensive (**interceptions**) and attacking (**assists**) contributions.
- Defenders (**CB**, **RB**, **LB**) contribute highly to **interceptions** and **tackles** showing key defensive contributions.


In [39]:
pivot_data = melted_data.pivot(index='Stat', columns='Position', values='Average Value')    # Pivot the data for heatmap to display average key stats by position

fig = go.Figure(    # Create a heatmap
    data=go.Heatmap(
        z=pivot_data.values,
        x=pivot_data.columns,
        y=pivot_data.index,
        colorscale='Viridis',
        colorbar=dict(title="Average Value"),
        text=pivot_data.round(2).values,  # Display values rounded to 2 decimal places on hover
        hoverinfo="text"  # Show values on hover
    )
)

for i, stat in enumerate(pivot_data.index): # Add annotations to heatmap
    for j, pos in enumerate(pivot_data.columns):
        fig.add_annotation(
            x=pos,
            y=stat,
            text=str(round(pivot_data.loc[stat, pos], 2)),  # Round to 2 decimal places
            showarrow=False,
            font=dict(color="white", size=10)
        )

fig.update_layout(
    title="Average Key Stats by Position",
    xaxis_title="Position",
    yaxis_title="Stat",
    xaxis=dict(tickmode='array', tickvals=pivot_data.columns),  # Display all positions on x-axis
    yaxis=dict(tickmode='array', tickvals=pivot_data.index) # Display all stats on y-axis
)

fig.show()
